# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [1]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [2]:
url = "https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_10.1.2025.pdf"
req = requests.get(url)

In [11]:
pdf = pdfplumber.open(BytesIO(req.content))

In [12]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [13]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [14]:
lines

[['Statewide Report by County - October 1, 2025',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'Inactive',
  'Total Active'],
 ['Aurora', '499', '2', '1,010', '233', '3', '100', '42', '1,847'],
 ['Beadle', '2,450', '29', '5,231', '1,373', '46', '1,215', '425', '10,344'],
 ['Bennett', '666', '8', '729', '190', '2', '201', '110', '1,796'],
 ['Bon Homme', '864', '10', '2,434', '490', '', '244', '173', '4,042'],
 ['Brookings',
  '5,129',
  '107',
  '10,224',
  '3,674',
  '23',
  '2,916',
  '835',
  '22,073'],
 ['Brown', '7,065', '103', '13,060', '3,587', '15', '2,044', '488', '25,874'],
 ['Brule', '873', '12', '1,770', '462', '4', '231', '87', '3,352'],
 ['Buffalo', '532', '2', '155', '79', '4', '101', '54', '873'],
 ['Butte', '754', '55', '4,672', '881', '22', '674', '895', '7,058'],
 ['Campbell', '103', '1', '909', '73', '', '51', '44', '1,137'],
 ['Charles Mix'

In [15]:
# report date is in the first line
report_date = lines.pop(0)

In [16]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2025-10-01


In [17]:
# save the PDF
outpath = Path('pdfs') / f'{report_date}.pdf'

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote pdfs/2025-10-01.pdf


In [18]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [19]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [20]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'inactive',
 'total active']

In [21]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [22]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [23]:
data[0]

{'county': 'Aurora',
 'democratic': 499,
 'libertarian': 2,
 'republican': 1010,
 'independent': 233,
 'other': 3,
 'npa': 100,
 'inactive': 42,
 'date': '2025-10-01'}

In [24]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [25]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/cjwinchester/south-dakota-voter-registration-data/data/2025-10-01.csv


In [26]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
